In [1]:
import os
import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA, LatentDirichletAllocation, TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
import pdfplumber  # Using pdfplumber instead of PyMuPDF
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import gensim
from gensim.corpora import Dictionary
from gensim.models import LsiModel
import warnings
import argparse

warnings.filterwarnings('ignore')

# Download necessary NLTK resources
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)


class PDFProcessor:
    """Extract text from PDF files using pdfplumber"""

    def __init__(self, directory_path):
        """
        Initialize with the directory containing PDF files

        Parameters:
        -----------
        directory_path : str
            Path to directory containing PDF files
        """
        self.directory_path = directory_path
        self.pdf_files = [f for f in os.listdir(directory_path) if f.endswith('.pdf')]
        self.documents = {}

    def extract_all_texts(self):
        """
        Extract text from all PDF files in the directory

        Returns:
        --------
        dict : Dictionary with filename as key and extracted text as value
        """
        for pdf_file in self.pdf_files:
            file_path = os.path.join(self.directory_path, pdf_file)
            self.documents[pdf_file] = self.extract_text(file_path)
        return self.documents

    @staticmethod
    def extract_text(file_path):
        """
        Extract text from a single PDF file using pdfplumber

        Parameters:
        -----------
        file_path : str
            Path to PDF file

        Returns:
        --------
        str : Extracted text
        """
        try:
            text = ""
            with pdfplumber.open(file_path) as pdf:
                for page in pdf.pages:
                    extracted = page.extract_text()
                    if extracted:
                        text += extracted + "\n"
            return text
        except Exception as e:
            print(f"Error extracting text from {file_path}: {e}")
            return ""


class TextPreprocessor:
    """Preprocess text data"""

    def __init__(self, additional_stopwords=None):
        """
        Initialize preprocessor with optional additional stopwords

        Parameters:
        -----------
        additional_stopwords : list, optional
            List of additional stopwords to remove
        """
        self.stop_words = set(stopwords.words('english'))
        if additional_stopwords:
            self.stop_words.update(additional_stopwords)
        self.lemmatizer = WordNetLemmatizer()

    def preprocess(self, text):
        """
        Preprocess a text document

        Parameters:
        -----------
        text : str
            Raw text to preprocess

        Returns:
        --------
        str : Preprocessed text
        """
        # Convert to lowercase
        text = text.lower()

        # Remove non-alphabetic characters and extra whitespace
        text = re.sub(r'[^a-zA-Z\s]', ' ', text)
        text = re.sub(r'\s+', ' ', text).strip()

        # Tokenize
        tokens = word_tokenize(text)

        # Remove stopwords and short tokens, lemmatize
        tokens = [self.lemmatizer.lemmatize(token) for token in tokens if
                  token not in self.stop_words and len(token) > 2]

        # Join back into a string
        return ' '.join(tokens)

    def preprocess_documents(self, documents):
        """
        Preprocess a dictionary of documents

        Parameters:
        -----------
        documents : dict
            Dictionary with document names as keys and raw text as values

        Returns:
        --------
        dict : Dictionary with document names as keys and preprocessed text as values
        """
        return {doc_name: self.preprocess(text) for doc_name, text in documents.items()}


class DocumentClustering:
    """Cluster documents into groups"""

    def __init__(self, n_clusters=2):
        """
        Initialize with the number of clusters

        Parameters:
        -----------
        n_clusters : int, optional
            Number of clusters to create (default is 2 for Accounting and Finance)
        """
        self.n_clusters = n_clusters
        self.vectorizer = TfidfVectorizer(max_features=5000, min_df=2, max_df=0.85)
        self.kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        self.document_names = None
        self.tfidf_matrix = None
        self.pca = PCA(n_components=2, random_state=42)
        self.labels = None

    def fit(self, preprocessed_documents):
        """
        Fit the clustering model to preprocessed documents

        Parameters:
        -----------
        preprocessed_documents : dict
            Dictionary with document names as keys and preprocessed text as values

        Returns:
        --------
        self : object
            Returns self
        """
        self.document_names = list(preprocessed_documents.keys())
        texts = [preprocessed_documents[doc] for doc in self.document_names]

        # Create TF-IDF matrix
        self.tfidf_matrix = self.vectorizer.fit_transform(texts)

        # Apply KMeans clustering
        self.labels = self.kmeans.fit_predict(self.tfidf_matrix)

        return self

    def get_cluster_terms(self, n_terms=20):
        """
        Get top terms for each cluster

        Parameters:
        -----------
        n_terms : int, optional
            Number of top terms to return

        Returns:
        --------
        dict : Dictionary with cluster IDs as keys and lists of top terms as values
        """
        feature_names = self.vectorizer.get_feature_names_out()
        cluster_terms = {}

        # Calculate cluster centroids
        order_centroids = self.kmeans.cluster_centers_.argsort()[:, ::-1]

        # Extract top terms for each cluster
        for cluster_id in range(self.n_clusters):
            top_term_indices = order_centroids[cluster_id, :n_terms]
            top_terms = [feature_names[i] for i in top_term_indices]
            cluster_terms[cluster_id] = top_terms

        return cluster_terms

    def identify_outliers(self, threshold=0.1):
        """
        Identify potential outlier documents that don't fit well in any cluster

        Parameters:
        -----------
        threshold : float, optional
            Distance threshold for identifying outliers

        Returns:
        --------
        list : List of potential outlier document names
        """
        # Calculate distances to assigned cluster center
        distances = np.min(
            [np.linalg.norm(self.tfidf_matrix.toarray() - center, axis=1)
             for center in self.kmeans.cluster_centers_],
            axis=0
        )

        # Identify outliers
        outlier_indices = np.where(distances > threshold)[0]
        outliers = [self.document_names[i] for i in outlier_indices]

        return outliers

    def get_cluster_documents(self):
        """
        Get documents assigned to each cluster

        Returns:
        --------
        dict : Dictionary with cluster IDs as keys and lists of document names as values
        """
        cluster_docs = {}
        for cluster_id in range(self.n_clusters):
            indices = np.where(self.labels == cluster_id)[0]
            cluster_docs[cluster_id] = [self.document_names[i] for i in indices]

        return cluster_docs

    def get_representative_documents(self):
        """
        Get the most representative document for each cluster (closest to centroid)

        Returns:
        --------
        dict : Dictionary with cluster IDs as keys and representative document names as values
        """
        representative_docs = {}

        for cluster_id in range(self.n_clusters):
            # Get indices of documents in this cluster
            cluster_indices = np.where(self.labels == cluster_id)[0]

            # Get documents in this cluster
            cluster_docs = [self.document_names[i] for i in cluster_indices]

            if len(cluster_docs) == 0:
                continue

            # Calculate distance to centroid for each document in cluster
            centroid = self.kmeans.cluster_centers_[cluster_id]
            cluster_vectors = self.tfidf_matrix[cluster_indices].toarray()

            # Find document closest to centroid
            distances = np.linalg.norm(cluster_vectors - centroid, axis=1)
            closest_idx = np.argmin(distances)

            # Get the document name
            representative_docs[cluster_id] = cluster_docs[closest_idx]

        return representative_docs


class SimilarityClassifier:
    """Classify documents based on similarity to cluster centers"""

    def __init__(self, vectorizer, kmeans_model, cluster_mapping=None):
        """
        Initialize with the vectorizer and KMeans model

        Parameters:
        -----------
        vectorizer : TfidfVectorizer
            Fitted vectorizer
        kmeans_model : KMeans
            Fitted KMeans model
        cluster_mapping : dict, optional
            Mapping from cluster IDs to subject names
        """
        self.vectorizer = vectorizer
        self.kmeans_model = kmeans_model
        self.cluster_mapping = cluster_mapping or {}
        self.threshold = 0.2  # Similarity threshold for classification

    def classify(self, text, preprocess_func=None):
        """
        Classify a new document

        Parameters:
        -----------
        text : str
            Text to classify
        preprocess_func : function, optional
            Function to preprocess the text

        Returns:
        --------
        tuple : (predicted_class, confidence_score, is_outlier)
        """
        if preprocess_func:
            text = preprocess_func(text)

        # Transform text to TF-IDF
        text_tfidf = self.vectorizer.transform([text])

        # Calculate distances to cluster centers
        distances = [np.linalg.norm(text_tfidf.toarray() - center)
                     for center in self.kmeans_model.cluster_centers_]

        # Find closest cluster
        closest_cluster = np.argmin(distances)
        min_distance = distances[closest_cluster]

        # Check if outlier
        is_outlier = min_distance > self.threshold

        # Map cluster to subject if mapping exists
        predicted_class = self.cluster_mapping.get(closest_cluster, f"Cluster {closest_cluster}")

        # Calculate confidence as inverse of normalized distance
        max_distance = max(distances)
        if max_distance > 0:
            confidence = 1 - (min_distance / max_distance)
        else:
            confidence = 1.0

        return predicted_class, confidence, is_outlier


class KeywordClassifier:
    """Classify documents based on accounting and finance keywords"""

    def __init__(self):
        """Initialize with accounting and finance keywords"""
        # Accounting keywords based on domain knowledge
        self.accounting_keywords = [
            'accounting', 'principles', 'standards', 'financial reporting', 'accounting standards',
            'financial statement', 'balance sheet', 'income statement', 'cash flow',
            'accounting', 'audit', 'ledger', 'journal entry', 'debit', 'credit',
            'accounts payable', 'accounts receivable', 'asset', 'liability', 'equity',
            'taxation', 'financial reporting', 'bookkeeping', 'accrual', 'depreciation',
            'amortization', 'inventory', 'cost accounting', 'budgeting', 'variance analysis',
            'profit', 'loss', 'revenue recognition', 'internal control', 'ifrs', 'gaap'
        ]

        # Finance keywords based on domain knowledge
        self.finance_keywords = [
            'investment', 'portfolio', 'risk', 'return', 'capital', 'valuation',
            'interest rate', 'bond', 'stock', 'market', 'security', 'option', 'futures',
            'derivative', 'dividend', 'corporate finance', 'capm', 'present value',
            'npv', 'irr', 'wacc', 'capital structure', 'leverage', 'beta', 'alpha',
            'financial market', 'efficient market', 'arbitrage', 'hedging', 'diversification',
            'financial management', 'merger', 'acquisition'
        ]

        # Syllabus keywords
        self.syllabus_keywords = [
            'syllabus', 'course outline', 'learning objective', 'prerequisite',
            'textbook', 'required reading', 'grading', 'assessment', 'assignment',
            'lecture', 'class schedule', 'course description', 'instructor', 'professor',
            'office hours', 'academic integrity', 'plagiarism', 'course policy',
            'attendance', 'participation', 'final exam', 'midterm'
        ]

    def is_syllabus(self, text):
        """
        Check if the document is likely a syllabus

        Parameters:
        -----------
        text : str
            Text to analyze

        Returns:
        --------
        bool : True if document is likely a syllabus, False otherwise
        """
        if text is None:
            return False

        text_lower = text.lower()
        # Count syllabus keywords
        syllabus_count = sum(text_lower.count(keyword) for keyword in self.syllabus_keywords)

        # If document contains at least 5 syllabus keywords, consider it a syllabus
        return syllabus_count >= 5

    def classify_by_filename(self, filename):
        """
        Classify document based on filename

        Parameters:
        -----------
        filename : str
            Name of the file to classify

        Returns:
        --------
        tuple : (predicted_class, confidence_score)
        """
        if filename is None:
            return "Unknown", 0.0

        filename_lower = filename.lower()

        accounting_indicators = ['acc', 'acct', 'accounting']
        has_accounting = any(indicator in filename_lower for indicator in accounting_indicators)

        finance_indicators = ['fin', 'finance']
        has_finance = any(indicator in filename_lower for indicator in finance_indicators)

        # After checking all syllabus, there is no file that is both accounting and finance
        if has_accounting:
            return "Accounting", 0.9
        elif has_finance:  # Syllabus whose filename contains finance might be an accounting syllabus
            return "Finance", 0.3  # So we impose a lower confidence
        else:
            return "Unknown", 0.0

    def classify(self, text, filename=None):
        """
        Classify text based on keyword frequency and filename if provided

        Parameters:
        -----------
        text : str
            Text to classify
        filename : str, optional
            Name of the file to classify

        Returns:
        --------
        tuple : (predicted_class, confidence_score)
        """
        if text is None:
            return "Unknown", 0.0

        text = text.lower()

        # Count accounting and finance keywords
        accounting_count = sum(text.count(keyword) for keyword in self.accounting_keywords)
        finance_count = sum(text.count(keyword) for keyword in self.finance_keywords)
        total_count = accounting_count + finance_count

        # Avoid division by zero
        if total_count == 0:
            keyword_class = "Unknown"
            keyword_confidence = 0.0
        elif accounting_count >= finance_count:
            keyword_confidence = accounting_count / (total_count or 1)  # Avoid division by zero
            keyword_class = "Accounting"
        else:
            keyword_confidence = finance_count / (total_count or 1)  # Avoid division by zero
            keyword_class = "Finance"

        # If no filename provided, use only keyword-based classification
        if filename is None:
            return keyword_class, keyword_confidence

        # Incorporate filename classification
        filename_class, filename_confidence = self.classify_by_filename(filename)

        if filename_class == keyword_class and filename_class != "Unknown":
            # Both methods agree, boost confidence
            confidence = (0.7 * keyword_confidence) + (0.3 * filename_confidence)
            return keyword_class, min(confidence, 0.95)
        elif filename_class != "Unknown":
            # Methods disagree, but filename gives a hint
            # Calculate combined confidence but weight keyword classification more
            confidence = (0.8 * keyword_confidence) + (0.2 * filename_confidence)
            return keyword_class, confidence
        else:
            # No useful info from filename, stick with keyword-based classification
            return keyword_class, keyword_confidence


class DocumentSimilarityClassifier:
    """Classify documents based on similarity to reference documents"""

    def __init__(self, reference_accounting_doc=None, reference_finance_doc=None):
        """
        Initialize with reference documents

        Parameters:
        -----------
        reference_accounting_doc : str, optional
            Reference accounting document text
        reference_finance_doc : str, optional
            Reference finance document text
        """
        self.reference_accounting_doc = reference_accounting_doc
        self.reference_finance_doc = reference_finance_doc
        self.preprocessor = TextPreprocessor()

    def calculate_similarity(self, text, reference_doc):
        """
        Calculate similarity between a document and a reference document

        Parameters:
        -----------
        text : str
            Text to analyze
        reference_doc : str
            Reference document text

        Returns:
        --------
        float : Similarity score
        """
        if not reference_doc:
            return 0.0

        # Preprocess both texts
        preprocessed_text = self.preprocessor.preprocess(text)
        preprocessed_reference = self.preprocessor.preprocess(reference_doc)

        # Create TF-IDF vectors
        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform([preprocessed_text, preprocessed_reference])

        # Calculate cosine similarity
        similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]

        return similarity

    def classify(self, text):
        """
        Classify text based on similarity to reference documents

        Parameters:
        -----------
        text : str
            Text to classify

        Returns:
        --------
        tuple : (predicted_class, confidence_score, similarity_scores)
        """
        # Calculate similarity to reference documents
        accounting_similarity = self.calculate_similarity(text, self.reference_accounting_doc)
        finance_similarity = self.calculate_similarity(text, self.reference_finance_doc)

        # Determine class based on highest similarity
        if accounting_similarity > finance_similarity:
            predicted_class = "Accounting"
            confidence = accounting_similarity
        else:
            predicted_class = "Finance"
            confidence = finance_similarity

        # Return both the classification and the raw similarity scores
        return predicted_class, confidence, (accounting_similarity, finance_similarity)


class TopicModeler:
    """Perform topic modeling using LSI and LDA"""

    def __init__(self, n_topics=2):
        """
        Initialize with number of topics

        Parameters:
        -----------
        n_topics : int, optional
            Number of topics to extract
        """
        self.n_topics = n_topics
        self.lda_model = LatentDirichletAllocation(
            n_components=n_topics,
            random_state=42,
            max_iter=10,
            learning_method='online'
        )
        self.lsi_model = None
        self.dictionary = None
        self.corpus = None
        self.vectorizer = TfidfVectorizer(max_features=5000, min_df=2, max_df=0.85)

    def fit_lda(self, preprocessed_documents):
        """
        Fit LDA model to preprocessed documents

        Parameters:
        -----------
        preprocessed_documents : dict
            Dictionary with document names as keys and preprocessed text as values

        Returns:
        --------
        self : object
            Returns self
        """
        texts = list(preprocessed_documents.values())
        self.tfidf_matrix = self.vectorizer.fit_transform(texts)
        self.lda_model.fit(self.tfidf_matrix)
        return self

    def fit_lsi(self, preprocessed_documents):
        """
        Fit LSI model to preprocessed documents

        Parameters:
        -----------
        preprocessed_documents : dict
            Dictionary with document names as keys and preprocessed text as values

        Returns:
        --------
        self : object
            Returns self
        """
        texts = list(preprocessed_documents.values())
        tokenized_texts = [text.split() for text in texts]

        # Create dictionary and corpus
        self.dictionary = Dictionary(tokenized_texts)
        self.corpus = [self.dictionary.doc2bow(text) for text in tokenized_texts]

        # Create LSI model
        self.lsi_model = LsiModel(
            self.corpus,
            id2word=self.dictionary,
            num_topics=self.n_topics
        )

        return self

    def get_lda_topics(self, n_terms=10):
        """
        Get top terms for each LDA topic

        Parameters:
        -----------
        n_terms : int, optional
            Number of top terms to return

        Returns:
        --------
        list : List of lists containing top terms for each topic
        """
        feature_names = self.vectorizer.get_feature_names_out()
        topics = []

        for topic_idx, topic in enumerate(self.lda_model.components_):
            top_terms_idx = topic.argsort()[:-n_terms - 1:-1]
            top_terms = [feature_names[i] for i in top_terms_idx]
            topics.append(top_terms)

        return topics

    def get_lsi_topics(self, n_terms=10):
        """
        Get top terms for each LSI topic

        Parameters:
        -----------
        n_terms : int, optional
            Number of top terms to return

        Returns:
        --------
        list : List of lists containing top terms for each topic
        """
        topics = []

        for topic_id in range(self.n_topics):
            top_terms = self.lsi_model.show_topic(topic_id, n_terms)
            topics.append([term for term, _ in top_terms])

        return topics

    def classify_with_lda(self, text, preprocess_func=None):
        """
        Classify a document using LDA topic distribution

        Parameters:
        -----------
        text : str
            Text to classify
        preprocess_func : function, optional
            Function to preprocess the text

        Returns:
        --------
        list : Topic distribution
        """
        if preprocess_func:
            text = preprocess_func(text)

        # Transform text to TF-IDF
        text_tfidf = self.vectorizer.transform([text])

        # Get topic distribution
        topic_dist = self.lda_model.transform(text_tfidf)[0]

        return topic_dist.tolist()

    def classify_with_lsi(self, text, preprocess_func=None):
        """
        Classify a document using LSI

        Parameters:
        -----------
        text : str
            Text to classify
        preprocess_func : function, optional
            Function to preprocess the text

        Returns:
        --------
        list : Topic distribution
        """
        if preprocess_func:
            text = preprocess_func(text)

        # Tokenize
        tokenized_text = text.split()

        # Convert to bow
        bow = self.dictionary.doc2bow(tokenized_text)

        # Get topic distribution
        topic_dist = self.lsi_model[bow]

        # Convert to dense representation
        dense_vec = np.zeros(self.n_topics)
        for topic_id, weight in topic_dist:
            dense_vec[topic_id] = weight

        return dense_vec.tolist()


class SyllabiAnalyzer:
    """Main class to analyze syllabi documents"""

    def __init__(self, directory_path):
        """
        Initialize with directory containing PDF files

        Parameters:
        -----------
        directory_path : str
            Path to directory containing PDF files
        """
        self.directory_path = directory_path
        self.pdf_processor = PDFProcessor(directory_path)
        self.text_preprocessor = TextPreprocessor()
        self.clustering = DocumentClustering()
        self.raw_documents = {}
        self.preprocessed_documents = {}
        self.similarity_classifier = None
        self.keyword_classifier = KeywordClassifier()
        self.topic_modeler = TopicModeler()
        self.reference_accounting_doc = None
        self.reference_finance_doc = None
        self.document_similarity_classifier = None

    def process_documents(self):
        """Process all documents in the directory"""
        self.raw_documents = self.pdf_processor.extract_all_texts()
        self.preprocessed_documents = self.text_preprocessor.preprocess_documents(self.raw_documents)
        return self

    def set_reference_documents(self, accounting_filename, finance_filename):
        """
        Set reference documents for similarity comparison

        Parameters:
        -----------
        accounting_filename : str
            Filename of the reference accounting document
        finance_filename : str
            Filename of the reference finance document

        Returns:
        --------
        self : object
            Returns self
        """
        accounting_path = os.path.join(self.directory_path, accounting_filename)
        finance_path = os.path.join(self.directory_path, finance_filename)

        self.reference_accounting_doc = PDFProcessor.extract_text(accounting_path)
        self.reference_finance_doc = PDFProcessor.extract_text(finance_path)

        # Initialize the similarity classifier with reference documents
        self.document_similarity_classifier = DocumentSimilarityClassifier(
            self.reference_accounting_doc,
            self.reference_finance_doc
        )

        return self

    def perform_clustering(self):
        """Perform document clustering"""
        self.clustering.fit(self.preprocessed_documents)

        # Initialize the similarity classifier with the clustering model
        self.similarity_classifier = SimilarityClassifier(
            self.clustering.vectorizer,
            self.clustering.kmeans
        )

        return self

    def analyze_clusters(self):
        """Analyze the formed clusters"""
        # Get top terms for each cluster
        cluster_terms = self.clustering.get_cluster_terms()

        # Get documents in each cluster
        cluster_docs = self.clustering.get_cluster_documents()

        # Identify potential outliers
        outliers = self.clustering.identify_outliers()

        # Get representative documents for each cluster
        representative_docs = self.clustering.get_representative_documents()

        # Map clusters to subjects based on terms
        cluster_mapping = {}
        for cluster_id, terms in cluster_terms.items():
            if any(term in ['financial statement', 'balance sheet', 'accounting'] for term in terms):
                cluster_mapping[cluster_id] = "Accounting"
            else:
                cluster_mapping[cluster_id] = "Finance"

        # Update similarity classifier with mapping
        self.similarity_classifier.cluster_mapping = cluster_mapping

        # If reference documents are not set, use representative documents
        if self.reference_accounting_doc is None or self.reference_finance_doc is None:
            accounting_cluster = next((k for k, v in cluster_mapping.items() if v == "Accounting"), None)
            finance_cluster = next((k for k, v in cluster_mapping.items() if v == "Finance"), None)

            if accounting_cluster is not None and finance_cluster is not None:
                accounting_doc = representative_docs.get(accounting_cluster)
                finance_doc = representative_docs.get(finance_cluster)

                if accounting_doc and finance_doc:
                    accounting_path = os.path.join(self.directory_path, accounting_doc)
                    finance_path = os.path.join(self.directory_path, finance_doc)

                    self.reference_accounting_doc = PDFProcessor.extract_text(accounting_path)
                    self.reference_finance_doc = PDFProcessor.extract_text(finance_path)

                    # Initialize the similarity classifier with reference documents
                    self.document_similarity_classifier = DocumentSimilarityClassifier(
                        self.reference_accounting_doc,
                        self.reference_finance_doc
                    )

        return cluster_mapping, outliers, cluster_terms, cluster_docs

    def perform_topic_modeling(self):
        """Perform topic modeling using LDA and LSI"""
        # Fit LDA model
        self.topic_modeler.fit_lda(self.preprocessed_documents)

        # Fit LSI model
        self.topic_modeler.fit_lsi(self.preprocessed_documents)

        # Get topics from LDA
        lda_topics = self.topic_modeler.get_lda_topics()

        # Get topics from LSI
        lsi_topics = self.topic_modeler.get_lsi_topics()

        return lda_topics, lsi_topics

    def evaluate_classification_methods(self, test_documents=None):
        """
        Evaluate different classification methods

        Parameters:
        -----------
        test_documents : dict, optional
            Dictionary with document names as keys and raw text as values

        Returns:
        --------
        dict : Evaluation results
        """
        # If no test documents provided, use a subset of existing documents
        if test_documents is None:
            test_docs = {}
            for i, (name, text) in enumerate(self.raw_documents.items()):
                if i % 5 == 0:  # Use every 5th document as test
                    test_docs[name] = text

            test_documents = test_docs

        results = {}

        for doc_name, raw_text in test_documents.items():
            preprocessed_text = self.text_preprocessor.preprocess(raw_text)

            # Check if it's a syllabus
            is_syllabus = self.keyword_classifier.is_syllabus(raw_text)
            if not is_syllabus:
                results[doc_name] = {
                    "is_syllabus": False,
                    "similarity": ("Not a Syllabus", 0.0, True),
                    "keyword": ("Not a Syllabus", 0.0),
                    "document_similarity": ("Not a Syllabus", 0.0, (0.0, 0.0)),
                    "lda": ("Not a Syllabus", 0.0),
                    "lsi": ("Not a Syllabus", 0.0)
                }
                continue

            # Similarity-based classification
            sim_class, sim_conf, is_outlier = self.similarity_classifier.classify(
                raw_text,
                preprocess_func=self.text_preprocessor.preprocess
            )

            # Keyword-based classification
            kw_class, kw_conf = self.keyword_classifier.classify(raw_text, doc_name)

            # Document similarity classification
            if self.document_similarity_classifier:
                doc_sim_class, doc_sim_conf, similarities = self.document_similarity_classifier.classify(raw_text)
            else:
                doc_sim_class, doc_sim_conf, similarities = "Unknown", 0.0, (0.0, 0.0)

/Users/yiningxiang/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
